## EU Call with sequantial pricing formla

Find the price of a EU call with $S_0 = K = 50$, $r = 0.1$ (annual), $\sigma = 0.4$,
and maturity in five months with the sequential pricing formula you
learned in this class.
Use step sizes of 1 month, 1 week (1/4 month), and 1 day (1/30 month).
Compare you solution with what you get from the matlab function
blsprice.

In [6]:
import numpy as np
from math import factorial as fact

S0 = 50
K = 50
r = 0.1
sigma = 0.4
T = 5/12

dm = 1/12
dw = 1/12 * 1/4
dd = 1/12 * 1/30

def EU_call_sequential(S0, K, r, T, dt, sigma):
    u = np.exp(sigma*np.sqrt(dt))
    d = np.exp(-sigma*np.sqrt(dt))
    p = (np.exp(r*dt) - d) / (u - d)
    n = int(T / dt)

    C = np.exp(-r*T) * sum([ (fact(n)/(fact(j)*fact(n-j)) * p**j * (1 - p)**(n-j) \
                               * max(0, u**j * d**(n-j) * S0 - K)) for j in range(n+1) ])
    
    return C

print(f"Price of EU call with step size of 1 month: {EU_call_sequential(S0, K, r, T, dm, sigma)}")
print(f"Price of EU call with step size of 1 week: {EU_call_sequential(S0, K, r, T, dw, sigma)}")
print(f"Price of EU call with step size of 1 day: {EU_call_sequential(S0, K, r, T, dd, sigma)}")

Price of EU call with step size of 1 month: 6.359545861058918
Price of EU call with step size of 1 week: 6.053260644489761
Price of EU call with step size of 1 day: 6.108025965389591


## EU and American call and put with binomial option pricing formula

Find the prices of a EU call and put, an American call and put, with $S_0 = K = 50$, $r = 0.1$ (annual), $\sigma = 0.4$, and maturity in five months with
the recursive binomial pricing formulas. \
Are the prices the same for EU and American options?

In [16]:
import numpy as np
from numba import njit

@njit
def binary_option_pricing(S0, K, r, T, subper, sigma, pos="call", op_type="EU"):
    ''' Calculate the price of a EU or an American option using the binomial method. 
        S0: Initial price of underlying.
        K: Strike price of option.
        r: Annual interest rate.
        T: Annual time period. Seven months = 7/12.
        subper: Number of subperiods in a year, e.g., 251 for 251 trading days in a year.
        sigma: Annual standard deviation.
        pos: Position of the option, whether it is a call or a put.
        op_type: Whether it is a EU or an American option.
    '''

    N = int(round(subper * T, 0)) # Trading steps from now to maturity (number of subperiods)
    dt = T / N

    u = np.exp(sigma*np.sqrt(dt))
    d = np.exp(-sigma*np.sqrt(dt))
    p = (np.exp(r*dt) - d)/(u - d)

    Price = np.zeros((N,N))
    S_tn = np.zeros((N,N))
    S_tn[0,0] = S0
    for it in range(1,N):
        for n in range(it+1):
            S_tn[it,n] = S0*(u**n)*(d**(it-n))
    
    # Check if call or put option and set the type variable, i.e., typ=-1: max(K - S, 0)
    if pos=="call": typ = 1 
    elif pos=="put": typ = -1
    else: 
        print("Please specify 'call' or 'put' for 'pos'") 
        return False
    
    for n in range(N):
            Price[N-1,n] = max(typ*(S_tn[N-1,n] - K),0)

    if op_type == "EU":
        for i in range(N-2,0,-1):
            for n in range(0,N-1):
                Price[i,n] = np.exp(-r*dt)*(p*Price[i+1,n+1] + (1-p)*Price[i+1,n])
    elif op_type == "american":
        for i in range(N-2,0,-1):
            for n in range(0,N-1):
                Price[i,n] = max(typ*(S_tn[i,n] - K), np.exp(-r*dt)*(p*Price[i+1,n+1] + (1-p)*Price[i+1,n]))
    else:
        print("Please specify 'EU' or 'american' for 'op_type'") 
        return False

    Price[0,0] = np.exp(-r*dt)*(p*Price[1,1] + (1-p)*Price[1,0])

    return Price[0,0]


S0 = 50
K = 50
r = 0.1
sigma = 0.4
T = 5/12
steps = 251*10

print(f"Price of EU call: {binary_option_pricing(S0, K, r, T, steps, sigma, 'call', 'EU')}")
print(f"Price of american call: {binary_option_pricing(S0, K, r, T, steps, sigma, 'call', 'american')}")
print(f"Price of EU put: {binary_option_pricing(S0, K, r, T, steps, sigma, 'put', 'EU')}")
print(f"Price of american put: {binary_option_pricing(S0, K, r, T, steps, sigma, 'put', 'american')}")


Price of EU call: 6.1143110558419895
Price of american call: 6.1143110558419895
Price of EU put: 4.075694380866513
Price of american put: 4.283653034089878
